# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [7]:
import pandas as pd
from patsy import dmatrices
import statsmodels.api as sm
import numpy as np

# Carregar os dados
df = pd.read_csv('previsao_de_renda.csv')

# Remover colunas desnecessárias
df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1, inplace=True)

# Tratar valores faltantes na coluna 'tempo_emprego' de forma mais segura
median_tempo_emprego = df['tempo_emprego'].median()
df['tempo_emprego'] = df['tempo_emprego'].fillna(median_tempo_emprego)

# Visualizar as primeiras linhas para conferir
print(df.head())

# Criando fórmula para o modelo com variáveis dummies
# As variáveis categóricas serão automaticamente tratadas com dummies
# A categoria mais frequente será mantida como referência utilizando o operador 'C()'
formula = 'np.log(renda) ~ ' + ' + '.join([f'C({col})' for col in df.select_dtypes(include='object').columns]) + ' + ' + ' + '.join(df.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist())

# Criar matrizes de design para o modelo
y, X = dmatrices(formula, df, return_type='dataframe')

# Ajustar o modelo de regressão
model = sm.OLS(y, X)
results = model.fit()

# Visualizar o sumário do modelo
print(results.summary())


  sexo  posse_de_veiculo  posse_de_imovel  qtd_filhos        tipo_renda  \
0    F             False             True           0        Empresário   
1    M              True             True           0       Assalariado   
2    F              True             True           0        Empresário   
3    F             False             True           1  Servidor público   
4    M              True            False           0       Assalariado   

            educacao estado_civil tipo_residencia  idade  tempo_emprego  \
0         Secundário     Solteiro            Casa     26       6.602740   
1  Superior completo       Casado            Casa     28       7.183562   
2  Superior completo       Casado            Casa     35       0.838356   
3  Superior completo       Casado            Casa     30       4.846575   
4         Secundário     Solteiro   Governamental     33       4.293151   

   qt_pessoas_residencia    renda  
0                    1.0  8060.34  
1                    2.0  

In [8]:
# Ajustar o modelo de regressão inicial
model = sm.OLS(y, X)
results = model.fit()

# Encontrar a variável com o maior valor p que não seja a constante
p_values = results.pvalues
p_values = p_values.drop('Intercept')  # Remover a intercepto se presente
max_p_value_var = p_values.idxmax()  # Variável com o maior p-value

print("Variável com o maior p-value:", max_p_value_var)
print("P-value:", p_values[max_p_value_var])

# Remover a variável menos significante
X_reduced = X.drop(columns=[max_p_value_var])

# Ajustar o modelo sem a variável menos significante
model_reduced = sm.OLS(y, X_reduced)
results_reduced = model_reduced.fit()

# Comparar os resultados
print("Sumário do modelo original:")
print(results.summary())
print("\nSumário do modelo reduzido:")
print(results_reduced.summary())


Variável com o maior p-value: C(educacao)[T.Superior completo]
P-value: 0.9658308622315754
Sumário do modelo original:
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.604
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                     911.7
Date:                Sun, 18 Aug 2024   Prob (F-statistic):               0.00
Time:                        21:28:43   Log-Likelihood:                -12486.
No. Observations:               15000   AIC:                         2.502e+04
Df Residuals:                   14974   BIC:                         2.522e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025

Avaliação Geral

A remoção da variável C(educacao)[T.Superior completo] do modelo parece apropriada, dado seu alto p-value e a inalteração nos indicadores principais de qualidade do modelo. Isso simplifica o modelo sem comprometer sua capacidade de explicar a variabilidade na renda logarítmica dos dados.

A análise sugere que o modelo reduzido é, ao menos, tão bom quanto o original com a vantagem de ser ligeiramente mais simples. Esta simplificação pode ajudar na interpretação e na generalização do modelo sem perder informação significativa.

In [10]:
import pandas as pd
from patsy import dmatrices
import statsmodels.api as sm
import numpy as np

# Carregar os dados
df = pd.read_csv('previsao_de_renda.csv')

# Remover colunas desnecessárias
df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1, inplace=True)

# Tratar valores faltantes na coluna 'tempo_emprego'
median_tempo_emprego = df['tempo_emprego'].median()
df['tempo_emprego'] = df['tempo_emprego'].fillna(median_tempo_emprego)

# Fórmula inicial
formula = 'np.log(renda) ~ ' + ' + '.join([f'C({col})' for col in df.select_dtypes(include='object').columns]) + ' + ' + ' + '.join(df.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist())

# Ajustar modelo inicial
y, X = dmatrices(formula, df, return_type='dataframe')
model = sm.OLS(y, X)
results = model.fit()

# Armazenar sumário do modelo inicial
initial_summary = results.summary()

# Remover variáveis com p-value maior que 0.05 iterativamente
significant = False
while not significant:
    p_values = results.pvalues
    p_values = p_values[p_values > 0.05]
    if not p_values.empty:
        max_p_value_var = p_values.idxmax()
        X = X.drop(columns=[max_p_value_var])
        model = sm.OLS(y, X)
        results = model.fit()
    else:
        significant = True

# Armazenar sumário do modelo final
final_summary = results.summary()

# Exibir os sumários para comparação
print("Sumário do modelo inicial:")
print(initial_summary)
print("\nSumário do modelo final:")
print(final_summary)


Sumário do modelo inicial:
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.604
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                     911.7
Date:                Sun, 18 Aug 2024   Prob (F-statistic):               0.00
Time:                        21:28:43   Log-Likelihood:                -12486.
No. Observations:               15000   AIC:                         2.502e+04
Df Residuals:                   14974   BIC:                         2.522e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

Indicadores Chave

    R-squared e Adjusted R-squared:
        Inicial: 0.604 e 0.603
        Final: 0.603 e 0.603
        Observação: Os valores de R² e Adjusted R² praticamente não mudaram, indicando que a quantidade de variância dos dados explicada pelo modelo permaneceu consistente, mesmo após a remoção das variáveis.

    AIC e BIC:
        Inicial: 25020 e 25220
        Final: 25010 e 25140
        Observação: Houve uma ligeira redução no AIC e no BIC, sugerindo uma melhoria na qualidade do ajuste do modelo ao considerar o número de parâmetros. Isso implica que o modelo final pode ser preferível devido à sua maior eficiência e simplicidade.

    F-statistic:
        Inicial: 911.7
        Final: 1424
        Observação: O aumento no F-statistic indica que a relação geral entre as variáveis independentes e a dependente permaneceu forte e até melhorou em termos de significância estatística global do modelo.

    P-values:
        Observação: Todas as variáveis remanescentes no modelo final têm p-values abaixo de 0.05, indicando que são estatisticamente significativas.

    Durbin-Watson:
        Inicial e Final: aproximadamente 2.012
        Observação: A estatística de Durbin-Watson manteve-se estável, sugerindo que não há autocorrelação residual significativa nos modelos.

Conclusão

A comparação dos modelos revela que o modelo final, apesar de ter menos variáveis, mantém praticamente a mesma capacidade de explicar a variabilidade dos dados em comparação com o modelo inicial. A redução das métricas AIC e BIC, junto com a manutenção dos outros indicadores de ajuste, sugere que o modelo final é mais parcimonioso sem sacrificar a capacidade explicativa.

O modelo final é, portanto, preferível para uso prático devido à sua simplicidade e eficácia equivalente. Isso ilustra um bom exemplo de como a simplificação do modelo pode ajudar na interpretação e na aplicação sem perda de informações importantes, alinhando-se aos princípios de um bom design de modelo estatístico.